In [ ]:
import pickle

In [ ]:
class Model:

    def init(self):
        pass

    def data_read(self, path):
        import pandas as pd
        import numpy as np
        self.df = pd.read_csv(path)
        return self.df.head(3)
    def preprocessing(self):
      print('Starting Data preprocessing')

      num_cols = self.df.select_dtypes(include=['int64', 'float64']).columns
      cat_cols = self.df.select_dtypes(include=['object']).columns
      from sklearn.preprocessing import LabelEncoder
      for col in cat_cols:
            le = LabelEncoder()
            self.df[col] = le.fit_transform(self.df[col])

      # using simple imputer
      from sklearn.impute import SimpleImputer
      si = SimpleImputer()
      self.df[num_cols]=si.fit_transform(self.df[num_cols])

      # get target columns
    def get_target_column(self):
        while True:
            target_col = input("Enter the target column name from the above list: ").strip()
            if target_col in self.df.columns:
                self.target_column = target_col
                print(f"Target column set to: {self.target_column}")
                break
            else:
                print("Invalid column name! Please enter a valid column.")
    # train test split
    def train_test_split(self, test_size=0.2, random_state=42):
         """Splits data into training and testing sets."""
         if self.target_column is None:
            print("Target column not set. Please run get_target_column() first.")
            return

         X = self.df.drop(columns=[self.target_column])
         y = self.df[self.target_column]
         from sklearn.model_selection import train_test_split
         self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state)
         from sklearn.preprocessing import StandardScaler
         self.sc = StandardScaler()
         self.X_train = self.sc.fit_transform(self.X_train)
         self.X_test = self.sc.transform(self.X_test)
    def train_naive_bayes(self):
        from sklearn.naive_bayes import GaussianNB , MultinomialNB , BernoulliNB
        """Trains a Gaussian Naïve Bayes classifier and evaluates it."""
        if self.X_train is None or self.y_train is None:
            print("Data not yet split. Run train_test_split() first.")
            return

        self.model = GaussianNB()
        self.model.fit(self.X_train, self.y_train)
        print("Naïve Bayes model trained.")

        # Make predictions
        y_pred = self.model.predict(self.X_test)

        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(self.y_test, y_pred)
        print(f"Model Accuracy: {accuracy:.2f}")

        from sklearn.metrics import precision_score
        precision = precision_score(self.y_test,y_pred)
        print(f"Model Precision :{precision:.2f}")
    import pickle
    def save_model(self, filename="model.pkl"):

        if self.model:
            with open(filename, 'wb') as file:
                pickle.dump({'model': self.model, 'scaler': self.sc}, file)
            print(f"Model saved successfully as {filename}")
        else:
            print("Model not trained yet! Train the model before saving.")

    def load_model(self, filename="model.pkl"):
        """Loads a trained model from a pickle file."""
        try:
            with open(filename, 'rb') as file:
                data = pickle.load(file)
                self.model = data['model']
                self.sc = data['scaler']
            print(f"Model loaded successfully from {filename}")
        except FileNotFoundError:
            print("No saved model found! Train and save a model first.")

In [ ]:
model = Model()
model.data_read('/content/Social_Network_Ads.csv')
model.preprocessing()
model.get_target_column()
model.train_test_split()
model.train_naive_bayes()
# Save the trained model
model.save_model("naive_bayes_model.pkl")

# Load the saved model in a new session (optional)
model.load_model("naive_bayes_model.pkl")


Starting Data preprocessing
Enter the target column name from the above list: Purchased
Target column set to: Purchased
Naïve Bayes model trained.
Model Accuracy: 0.91
Model Precision :0.89
Model saved successfully as naive_bayes_model.pkl
Model loaded successfully from naive_bayes_model.pkl


In [ ]:
with open("/content/naive_bayes_model.pkl", "rb") as file:
    data = pickle.load(file)
    model = data["model"]
    scaler = data["scaler"]